In [35]:
'''
今日头条，终于爬取下来了
1.webdriver获取分析来的url，此url当然是ajax,network-xhr
2,获取的是json格式的数据，分析清理掉多余的部分，再通过json.loads-python对象的数据
3.jsonpath.jsonpath过得所有图片url，并分析，提取，清理所需要的
4.存在本地，以数字.jpg的格式。

'''
from selenium import webdriver
import json
import jsonpath
import requests
driver = webdriver.Chrome()
driver.get('https://www.toutiao.com/search_content/?offset=0&format=json&keyword=%E9%A3%8E%E6%99%AF&autoload=true&count=20&cur_tab=1&from=search_tab')
html = driver.page_source
jd = html.lstrip('<htmlxmlns="http://www.w3.org/1999/xhtml"><head></head><body><prestyle="word-wrap: break-word; white-space: pre-wrap;">').rstrip('</pre></body></html>')
a = json.loads(jd)
image = []
images = jsonpath.jsonpath(a,'$.data..image_list[*]')
for i in images:
    image.append(i)
#print(image)
url = []
for i in image:
    if isinstance(i,dict):
        url.append(i['url'])
    else:
        url.append(i.lstrip('http:'))
        
#print(url)
file_path = "F://toutiaofengjing//"
x = 1
for u in url:
    data = requests.get("http:"+u)
    photo_name = '{}.jpg'.format(x)
    x+=1
    path = file_path+photo_name
    f=open(path,'wb')
    f.write(data.content)
    f.close()
    

In [2]:
'''
网上copy的例子爬取的不全。？？
'''
import requests
from urllib.parse import urlencode
import os
from hashlib import md5
import json


def get_page(offset,keyword):
    params={
        'offset':offset,
        'format':'json',
        'keyword':keyword,
        'autoload':'true',
        'count':'20',
        'cur_tab':'1',
        'from':'search_tab'
    }
    #https://www.toutiao.com/search_content/?offset=60&format=json&keyword=%E8%BD%A6%E6%A8%A1&autoload=true&count=20&cur_tab=1&from=search_tab
    url='https://www.toutiao.com/search_content/?'+urlencode(params)

    response=requests.get(url)
    #500服务器内部错误，400错误请求（服务器找不到请求的语法） 404未找到
    if response.status_code==200:
        return response.json()

def get_images(json):
    data=json.get('data')
    if data:
        for item in data:
            image_list=item.get('image_list')
            title=item.get('title')

            if image_list:
                for image in image_list:
                
                    #构造一个生成器，将图片和标题一起返回
                    yield {
                        'image':image.get('url'),
                        'title':title
                    }
#item就是get_image()返回的一个字典
#item里面的title创建一个文件夹
def save_image(item):
    if not os.path.exists(item.get('title')):
        os.mkdir(item.get('title'))

    local_image_url=item.get('image')
    response=requests.get("http:"+local_image_url)

    if response.status_code==200:
        file_path='{0}/{1}.{2}'.format(item.get('title'),md5(response.content).hexdigest(),'jpg')

        #判断路径是否存在，如果不存在，写入
        if not os.path.exists(file_path):
            with open(file_path,'wb')as f:
                f.write(response.content)



#定义一个offset数组，遍历，提取图片，下载
def main(offset,keyword):
    json=get_page(offset,keyword)
    for item in get_images(json):
        print(item)
        save_image(item)
if __name__ == '__main__':
    keyword=input("请输入要爬取图片的关键词:")
    offset=input("请输入要爬取的数量:")
    main(offset,keyword)


请输入要爬取图片的关键词:街拍
请输入要爬取的数量:20
{'image': '//p3.pstatp.com/list/pgc-image/153702541412411e843442e', 'title': '街拍成都：初秋来了，成熟的时尚穿搭，让你魅力无限！'}
{'image': '//p9.pstatp.com/list/pgc-image/1537025414063ef0382dcb7', 'title': '街拍成都：初秋来了，成熟的时尚穿搭，让你魅力无限！'}
{'image': '//p3.pstatp.com/list/pgc-image/1537025414124c3a8f4957e', 'title': '街拍成都：初秋来了，成熟的时尚穿搭，让你魅力无限！'}
{'image': '//p3.pstatp.com/list/pgc-image/15370254142658d81f137c9', 'title': '街拍成都：初秋来了，成熟的时尚穿搭，让你魅力无限！'}
{'image': '//p1.pstatp.com/list/pgc-image/15370233514373b062df16f', 'title': '路人街拍，美的令人神往的秋季搭配，让你成为全世界的女神！'}
{'image': '//p1.pstatp.com/list/pgc-image/15370233517455d5b434bd2', 'title': '路人街拍，美的令人神往的秋季搭配，让你成为全世界的女神！'}
{'image': '//p3.pstatp.com/list/pgc-image/15370233519320978892423', 'title': '路人街拍，美的令人神往的秋季搭配，让你成为全世界的女神！'}
{'image': '//p1.pstatp.com/list/pgc-image/1537023352053dc88552085', 'title': '路人街拍，美的令人神往的秋季搭配，让你成为全世界的女神！'}
{'image': '//p9.pstatp.com/list/pgc-image/153702336438777e7c83c32', 'title': '路人街拍，就算有女神范也离不开时尚气质的穿搭'}
{'image'

AttributeError: 'str' object has no attribute 'get'